# Pràctica 2 - Creació de la visualització i lliurament del projecte

## Aarón Puche Benedito

In [1]:
import pandas as pd

Carreguem i veiem les dades originals:

In [2]:
df = pd.read_csv('data/franquisme_1938_1978.csv')
df.head()

,Codi,Cognoms nom,Cognoms,Nom,Sexe,Edat,Municipi naixement,Pedanies/Agregats naixement,Comarca naixement,Província naixement,...,Commutació/indult (demanat),Afusellades,Ref num arxiu,Autoria de la descripció,Data de la descripció,Data correccio registre,Municipi_Naixement_Longitud_ETRS89,Municipi_Naixement_Latitud_ETRS89,Municipi_Residencia_Longitud_ETRS89,Municipi_Residencia_Latitud_ETRS89
0,135218,"ABAD ALFONSO, Erundina",ABAD ALFONSO,Erundina,Dona,43.0,Alcoi,--,Alcoià,Alacant,...,NaN,NaN,37729.0,ANC,2017-07-12,NaN,-0.473184,38.698369,2.176349,41.384247
1,25335,"ABAD ARBÓS, Ricardo",ABAD ARBÓS,Ricardo,Home,30.0,Barcelona,--,Barcelonès,Barcelona,...,NaN,NaN,12842.0,ANC,2017-07-12,NaN,2.176349,41.384247,2.176349,41.384247
2,123192,"ABAD BARAS, José",ABAD BARAS,José,Home,30.0,Benavarri,--,Ribagorça,Osca,...,NaN,NaN,49476.0,ANC,2017-07-12,NaN,0.481881,42.106994,0.620619,41.615274
3,171,"ABAD BATLLONE, Juan",ABAD BATLLONE,Juan,Home,42.0,Barcelona,--,Barcelonès,Barcelona,...,NaN,NaN,4063.0,ANC,2017-07-12,NaN,2.176349,41.384247,2.176349,41.384247
4,24485,"ABAD BOIRA, Ricardo",ABAD BOIRA,Ricardo,Home,48.0,Tauste,--,--,Saragossa,...,NaN,NaN,12915.0,ANC,2017-07-12,NaN,-1.256941,41.921394,2.176349,41.384247


In [3]:
df.shape

(69832, 34)

In [4]:
df.columns

Index(['Codi', 'Cognoms nom', 'Cognoms', 'Nom', 'Sexe', 'Edat',
       'Municipi naixement', 'Pedanies/Agregats naixement',
       'Comarca naixement', 'Província naixement',
       'Comunitat autònoma naixement', 'País naixement', 'Municipi residència',
       'Pedanies/Agregats residencia', 'Comarca residència',
       'Província residència', 'Comunitat autònoma residència',
       'País residència', 'Tipus procediment 1', 'Tipus procediment 2',
       'Num causa', 'Any inicial', 'Any aprovació sen o altra resol', 'Pena',
       'Commutació/indult (demanat)', 'Afusellades', 'Ref num arxiu',
       'Autoria de la descripció', 'Data de la descripció',
       'Data correccio registre', 'Municipi_Naixement_Longitud_ETRS89',
       'Municipi_Naixement_Latitud_ETRS89',
       'Municipi_Residencia_Longitud_ETRS89',
       'Municipi_Residencia_Latitud_ETRS89'],
      dtype='object')

### Pregunta 1: En quines comarques hi ha hagut més repressió?

Per a respondre a aquestes dades hem decidit realitzar una visualització amb un mapa i punts sobre ell.

Per a respondre a aquestes dades hem decidit realitzar una visualització amb un mapa. En concret ho farem amb un mapa de Catalunya dividit per les comarques. D'aquesta manera veurem de forma molt intuïtiva quin ha sigut el nivell de repressió per comarca.

El que farem és veure el nombre total de persones que han sigut sentenciades amb alguna pena al llarg dels anys per comarca. Per a fer això necessitem saber a quina comarca pertany cada cas i l'any de la seua sentència.

L'any de la sentència ho tenim clar, ja que és el camp *Any aprovació sen o altra resol*. Però per decidir la comarca ala que pertany la persona tenim dues opcions, agafar les dades de naixement o les dades de residència. Hem cregut que serà millor agafar les dades de residència, ja que creguem que és on la persona estaria vivint quan va ser jutjada.

Veiem quins llocs de residència tenim:

In [5]:
df["Comunitat autònoma naixement"].unique()

array(['Comunitat Valenciana', 'Catalunya', 'Aragó', 'Castella-Lleó',
       '--', 'Cantàbria', 'Regió de Múrcia', 'Andalusia', 'Illes Balears',
       'Comunitat de Madrid', 'País Basc', 'Galícia', 'Melilla',
       'Castella-la Manxa', 'Extremadura', "Principat d'Astúries",
       'Comunitat Foral de Navarra', 'Rioja', 'Illes Canàries',
       'Castella i Lleó', 'Ceuta', 'Madrid'], dtype=object)

Les dades corresponen a les persones a les quals es va instruir un procediment judicial militar durant la dictadura de Franco pels tribunals de l'Auditoria de Guerra de l'Exèrcit d'Ocupació que van actuar a Catalunya de l'abril de 1938 a desembre de 1978.

Com veiem, trobem persones que residien en altres comunitats a part de Catalunya. La idea inicial per respondre a la pregunta plantejada era visualitzar un mapa de Catalunya per comarques, si tenim dades d'altres comunitats, és possible que hàgem de replantejar-nos aquesta solució.

Abans de canviar de plantejament, veiem el total de casos de persones que residien fora de Catalunya:

In [6]:
df[df["Comunitat autònoma residència"] != "Catalunya"].shape

(3307, 34)

Tenim unes 3307 persones. Si comparem amb el total que són unes 70.000, podem veure que és un percentatge bastant menut, per tant, no tindrem en conter aquests casos i seguirem amb el nostre plantejament inicial.

Ens quedem sols amb els casos de persones que residien a Catalunya:

In [7]:
df_catalunya = df[df["Comunitat autònoma residència"] == "Catalunya"]

# Comprovem
print(df_catalunya.shape)
df_catalunya["Comunitat autònoma residència"].unique()

(66525, 34)


array(['Catalunya'], dtype=object)

Comprovem ara si hi ha valors buits en el camp *Comarca residència*:

In [8]:
df_catalunya["Comarca residència"].isnull().sum()

0

In [9]:
df_catalunya["Comarca residència"].isna().sum()

0

No hi ha cap valor *null* ni *na*.

Comprovem també si hi ha algun valor especial per a indicar aquest aspecte:

In [10]:
df_catalunya["Comarca residència"].unique()

array(['Barcelonès', 'Segrià', 'Baix Camp', 'Vallès Occidental', 'Bages',
       'Maresme', 'Anoia', 'Berguedà', 'Montsià', "Val d'Aran", 'Osona',
       'Priorat', 'Tarragonès', 'Noguera', 'Baix Llobregat',
       'Vallès Oriental', 'Pallars Sobirà', 'Garrigues', 'Ripollès',
       'Baix Ebre', 'Gironès', "Ribera d'Ebre", 'Baix Empordà',
       'Conca de Barberà', 'Pallars Jussà', 'Urgell', 'Garraf',
       'Cerdanya', 'Garrotxa', 'Selva', "Pla de l'Estany", 'Segarra',
       "Pla d'Urgell", 'Alt Penedès', 'Terra Alta', 'Alt Camp',
       'Alt Empordà', 'Alta Ribagorça', 'Baix Penedès', 'Alt Urgell',
       'Solsonès', 'Moianès', 'Baix Penedès / Maresme',
       'Barcelonès / Vallès Oriental', 'Barcelonès / np',
       'Barcelonès / Maresme', 'Anoia / Alt Penedès / Baix Llobregat',
       'Barcelonès / Osona', '--', 'Barcelonès / Tarragonès',
       'Baix Llobregat / Barcelonès', 'Osona / Ripollès',
       'Baix Ebre / Barcelonès', 'Barcelonès / Vallès Occidental'],
      dtype=object

Veiem que hi ha alguns valors on hi ha indicada més d'una comarca, comprovem si són molts:

In [11]:
df_catalunya[df_catalunya["Comarca residència"].str.contains("/")].shape

(19, 34)

Sols són 19, així que els eliminem:

In [12]:
df_catalunya = df_catalunya[(df_catalunya["Comarca residència"].str.contains("/")) == False]

Podem veure també el valor '--'. Aquest valor indicara que no res coneix la comarca de residència.

Comprovem si hi ha molts casos:

In [13]:
df_catalunya[df_catalunya["Comarca residència"] == "--"].shape

(5, 34)

Sols hi ha 5 casos, passem a eliminar-los:

In [14]:
df_catalunya = df_catalunya[df_catalunya["Comarca residència"] != "--"]

# Comprovem
df_catalunya[df_catalunya["Comarca residència"] == "--"].shape

(0, 34)

En aquest punt ja tenim tractat el camp *Comarca residència*.

Mirem ara els anys de quan es van executar les distintes sentències:

In [15]:
df_catalunya["Any aprovació sen o altra resol"].unique()

array([1943., 1939., 1942., 1940.,   nan, 1941., 1959., 1969., 1948.,
       1938., 1946., 1951., 1944., 1945., 1949., 1950., 1952., 1961.,
       1963., 1958., 1955., 1957., 1972., 1954., 1970., 1947., 1953.,
       1968., 1960., 1975., 1978., 1956., 1979., 1962., 1977., 1967.,
       1971., 1965., 1973., 1964., 1976., 1966., 1974., 1980.])

Contem el total de valors *na* en aquest camp:

In [16]:
df_catalunya["Any aprovació sen o altra resol"].isna().sum()

3773

Tenim un total de 3773 camps sense any d'execució.

Eliminem aquests registres:

In [17]:
df_catalunya.dropna(subset = ["Any aprovació sen o altra resol"], inplace=True)

#Comprovem
df_catalunya["Any aprovació sen o altra resol"].isna().sum()

0

En aquest punt ja tenim els camps de comarques i anys tractats.

Hem eliminat al voltant de 6000 registres que no complien amb els nostres requisits. En comparació al volum total de dades no és molt, no representa ni el 10% del total.

Ara creem el dataset per a la visualització. El que farem serà contar el total de casos per any i comarca.

In [18]:
# Creem un dataset buit
df_mapa = pd.DataFrame()

# Creem les variables comarques i anys_sentencia que utilitzarem després
comarques = df_catalunya["Comarca residència"].sort_values().unique()
anys_sentencia = df_catalunya["Any aprovació sen o altra resol"].sort_values().unique()

# Afegim la columna comarca
# Són les comarques úniques ordenades alfabèticament
df_mapa["comarca"] = df_catalunya["Comarca residència"].sort_values().unique()

# Comprovem
df_mapa.head()

,comarca
0,Alt Camp
1,Alt Empordà
2,Alt Penedès
3,Alt Urgell
4,Alta Ribagorça


Ara recorrerem cada any i calcularem el total de casos per any i comarca:

In [19]:
for a in anys_sentencia:
    df_mapa[int(a)] = ""
    for c in comarques:
        df_mapa[a][df_mapa["comarca"] == c] = df_catalunya[(df_catalunya["Any aprovació sen o altra resol"] == a) & (df_catalunya["Comarca residència"] == c)].shape[0]

# Comprovem        
df_mapa.head()

,comarca,1938,1939,1940,1941,1942,1943,1944,1945,1946,...,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980
0,Alt Camp,0,450,85,31,26,50,5,2,0,...,0,0,0,0,0,0,0,0,0,0
1,Alt Empordà,1,715,246,51,113,105,14,28,35,...,0,0,0,0,0,0,0,0,0,0
2,Alt Penedès,1,541,168,61,149,163,7,3,2,...,0,0,0,0,0,0,0,0,0,0
3,Alt Urgell,22,155,113,32,42,29,3,15,2,...,0,0,0,0,0,2,0,0,2,0
4,Alta Ribagorça,78,9,11,3,5,5,0,3,0,...,0,0,0,0,0,0,0,0,0,0


Exportem el dataset en un csv:

In [20]:
df_mapa.to_csv('data/dades_mapa.csv', index=False)

### Pregunta 2: Hi ha diferències significatives entre les característiques de les víctimes depenent del sexe?

Veient els atributs que tenim de les víctimes, agafarem la comunitat de residència i l'edat per veure si hi ha alguna diferència significativa en la repressió que van sofrir les persones que vivien en una mateixa província o tingueren una mateixa edat. Farem una distinció per sexe i així tindrem present una perspectiva de gènere. D'aquesta manera comprovarem si en algunes províncies o rangs d'edat la repressió ha estat més present en homes o dones.

Volem fer una comparació relativa, per tant, utilitzarem un gràfic de barres apilades al 100%. En concret farem servir dos gràfics d'aquests, un agrupant les víctimes per edat i altre per província.

Elegim la província, ja que si fem la comparació per comarques o municipis hi haurà massa columnes i açò és una limitació d'aquest tipus de gràfics. Seria molt difícil obtenir informació útil amb eixa visualització.

Pel que fa als anys, els dividirem en rangs pel mateix motiu.

Comencem preparant les dades per a la visualització del gràfic per províncies de residència. Partirem del dataset de Catalunya que ja tenim preparat.

Comprovem que tenim les províncies correctes:

In [21]:
df_catalunya["Província residència"].unique()

array(['Barcelona', 'Lleida', 'Tarragona', 'Girona'], dtype=object)

Tractem també el camp *Edat* per veure quins valors té:

In [22]:
df_catalunya["Edat"].sort_values().unique()

array([  12.,   13.,   14.,   15.,   16.,   17.,   18.,   19.,   20.,
         21.,   22.,   23.,   24.,   25.,   26.,   27.,   28.,   29.,
         30.,   31.,   32.,   33.,   34.,   35.,   36.,   37.,   38.,
         39.,   40.,   41.,   42.,   43.,   44.,   45.,   46.,   47.,
         48.,   49.,   50.,   51.,   52.,   53.,   54.,   55.,   56.,
         57.,   58.,   59.,   60.,   61.,   62.,   63.,   64.,   65.,
         66.,   67.,   68.,   69.,   70.,   71.,   72.,   73.,   74.,
         75.,   76.,   77.,   78.,   79.,   80.,   81.,   82.,   84.,
         85.,   89., 9999.,   nan])

Tenim edats entre 12 i 89 anys.

I també veiem que tenim valors *na* i algun valor 9999 que significarà que no tenim eixe valor.

Comprovem el total de casos amb aquests valors:

In [23]:
df_catalunya["Edat"].isna().sum()

2

In [24]:
df_catalunya[df_catalunya["Edat"] == 9999].shape

(1876, 34)

Són un total de 1878 casos. Podem eliminar-los i continuarem tenim més de 60.000 casos en el dataset.

In [25]:
df_catalunya.dropna(subset = ["Edat"], inplace=True)
df_catalunya = df_catalunya[df_catalunya["Edat"] != 9999]

# Comprovem
print(df_catalunya["Edat"].isna().sum())
df_catalunya[df_catalunya["Edat"] == 9999].shape

0


(0, 34)

Creem ara un nou dataset i l'omplim amb les dades ja preparades:

In [26]:
# Creem un dataset buit
df_column_provincies = pd.DataFrame()

# Afegim les províncies
provincies = df_catalunya["Província residència"].unique()
df_column_provincies["provincia"] = provincies

# Afegim les columnes homes i dones
df_column_provincies["homes"] = ""
df_column_provincies["dones"] = ""

# Ara afegim el total d'homes i dones represaliades en cada província
for p in provincies:
    df_column_provincies["homes"][df_column_provincies["provincia"] == p] = df_catalunya[(df_catalunya["Província residència"] == p) & (df_catalunya["Sexe"] == "Home")].shape[0]
    df_column_provincies["dones"][df_column_provincies["provincia"] == p] = df_catalunya[(df_catalunya["Província residència"] == p) & (df_catalunya["Sexe"] == "Dona")].shape[0]


# Comprovem
df_column_provincies.head()

,provincia,homes,dones
0,Barcelona,33330,3544
1,Lleida,8359,538
2,Tarragona,8387,608
3,Girona,5855,229


Guardem el resultat:

In [27]:
df_column_provincies.to_csv('data/dades_columnes_provincies.csv', index=False)

Ara fem una cosa similar però amb les edats. Les dividirem en rangs per no tenir una columna per cada edat i no saturar el gràfic, fent difícil la seua comprensió.

In [28]:
# Creem un dataset buit
df_column_edat = pd.DataFrame()

# Omplim la columna anys
edats = 10,19,20,29,30,39,40,49,50,59,60,69,70,79,80,89
rang_edats = []
i = 0
while i < len(edats)-1:
    rang_edats.append(str(edats[i])+"-"+str(edats[i+1])+" anys")
    i +=2

df_column_edat["edats"] = rang_edats

# Creem les columnes homes i dones
df_column_edat["homes"] = ""
df_column_edat["dones"] = ""

# Dividim tots els casos que tenim per aquests rangs d'edat i gènere
i = 0
while i < len(edats)-1:
    rang = str(edats[i])+"-"+str(edats[i+1])+" anys"
    df_column_edat["homes"][df_column_edat["edats"] == rang] = df_catalunya[(df_catalunya["Edat"].between(edats[i],edats[i+1])) & (df_catalunya["Sexe"] == "Home")].shape[0]
    df_column_edat["dones"][df_column_edat["edats"] == rang] = df_catalunya[(df_catalunya["Edat"].between(edats[i],edats[i+1])) & (df_catalunya["Sexe"] == "Dona")].shape[0]
    i +=2
    
# Comprovem
df_column_edat  

,edats,homes,dones
0,10-19 anys,1183,254
1,20-29 anys,13121,1451
2,30-39 anys,18476,1326
3,40-49 anys,13516,1062
4,50-59 anys,7201,547
5,60-69 anys,2137,234
6,70-79 anys,279,44
7,80-89 anys,18,1


Guardem el resultat:

In [29]:
df_column_edat.to_csv('data/dades_columnes_edats.csv', index=False)

### Pregunta 3: Com ha evolucionat el nombre d'afusellaments al llarg de la dictadura?

En aquesta pregunta el que volem saber és la tendència en el nombre d'afusellaments al llarg de la dictadura, per tant, hem decidit utilitzar un gràfic de línies. En la PAC 3 hem fet servir aquest gràfic i ens ha sigut realment útil per veure de forma ràpida i senzilla l'evolució dels salaris i dels preus de lloguer a Catalunya.

Per tractar aquestes dades primer agafarem aquells casos on la persona ha sigut afusellada. Com en la pregunta anterior, partirem de les dades que ja tenim tractades en el dataset de Catalunya.

In [30]:
df_afusellats = df_catalunya[df_catalunya["Afusellades"] == "executat/da"]
df_afusellats.shape

(3233, 34)

Tenim un total de 3233 persones que van ser afusellades.

El que farem ara és crear el dataset amb les dades d'aquestes persones. Ens quedarem sols amb les sentències fins a l'any 1975, que va ser quan va morir Franco.

Calcularem el total de persones afusellades i també distingirem per sexe com hem fet en l'exercici anterior per comprovar si se segueix la mateixa tendència.

Primer, comprovem els anys dels afusellaments:

In [31]:
df_afusellats["Any aprovació sen o altra resol"].sort_values().unique()

array([1938., 1939., 1940., 1941., 1942., 1943., 1944., 1945., 1946.,
       1947., 1948., 1949., 1950., 1952., 1954., 1956., 1974., 1975.])

No serà necessari filtrar les dades per any, l'any màxim que tenim ja és el 1975.

Creem ara el dataset amb les dades preparades:

In [32]:
# Creem el dataset buit
df_column_afusellaments = pd.DataFrame()

# Agafem els anys de què disposem de dades
anys_sentencia = df_afusellats["Any aprovació sen o altra resol"].sort_values().unique()

# Omplim la columna anys
df_column_afusellaments["anys"] = list(map(int, anys_sentencia))

# Inicialitzem les columnes per a les dades
df_column_afusellaments["total"] = 0
df_column_afusellaments["homes"] = 0
df_column_afusellaments["dones"] = 0

# Omplim aquestes noves columnes creades
for a in anys_sentencia:
    df_column_afusellaments["total"][df_column_afusellaments["anys"] == a] = df_afusellats[df_afusellats["Any aprovació sen o altra resol"] == a].shape[0]
    df_column_afusellaments["homes"][df_column_afusellaments["anys"] == a] = df_afusellats[(df_afusellats["Any aprovació sen o altra resol"] == a) & (df_afusellats["Sexe"] == "Home")].shape[0]
    df_column_afusellaments["dones"][df_column_afusellaments["anys"] == a] = df_afusellats[(df_afusellats["Any aprovació sen o altra resol"] == a) & (df_afusellats["Sexe"] == "Dona")].shape[0]


df_column_afusellaments

,anys,total,homes,dones
0,1938,59,59,0
1,1939,2316,2302,14
2,1940,455,454,1
3,1941,77,77,0
4,1942,132,132,0
5,1943,138,137,1
6,1944,13,13,0
7,1945,1,1,0
8,1946,3,3,0
9,1947,8,8,0


Guardem les dades:

In [33]:
df_column_afusellaments.to_csv('data/dades_linia_afusellaments.csv', index=False)